In [2]:
import pandas as pd
import numpy as np
import os
import tqdm

from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib notebook

# Leitura do banco

In [26]:
data_dir = "./"
db_file = 'fundos.db'
engine = create_engine(
    "sqlite:///"+os.path.abspath(os.path.join(data_dir, db_file)))

data_inicio = '1500-02-26'
data_fim = '2018-12-31'
cnpj_fundo = '12.055.107/0001-16'  # ALASK BLACK

In [12]:
# carteira e pesos
sqlquery = """
select
    CNPJ_FUNDO,
    DENOM_SOCIAL,
    DT_COMPTC,
    FILE,
    VL_PATRIM_LIQ,
    VL_MERC_POS_FINAL,
    TP_APLIC,
    TP_ATIVO,
    DS_ATIVO
    
from cda
where DT_COMPTC >= '{data_inicio}'
and DT_COMPTC <= '{data_fim}'
""".format(data_inicio=data_inicio, data_fim=data_fim)
carteira = pd.read_sql_query(sqlquery, engine, parse_dates=['DT_COMPTC'])
#pl = carteira.sort_values(['DT_COMPTC', 'FILE'])['VL_PATRIM_LIQ'].fillna(method='bfill')
#carteira['VL_PATRIM_LIQ'] = pl
#carteira['peso'] = carteira['VL_MERC_POS_FINAL'].div(carteira['VL_PATRIM_LIQ'])

#carteira = carteira[carteira.FILE != 'PL']

# Valores a pagar são negativos
q = (carteira.TP_APLIC == 'Valores a pagar') | (carteira.TP_APLIC.str.lower().str.contains('obriga|lançad'))
carteira.loc[q, ['VL_MERC_POS_FINAL', 'peso']] *= -1

In [31]:
# carteira e pesos
sqlquery = """
select
    CNPJ_FUNDO,
    DENOM_SOCIAL,
    DT_COMPTC,
    FILE,
    VL_PATRIM_LIQ,
    VL_MERC_POS_FINAL,
    TP_APLIC,
    TP_ATIVO,
    DS_ATIVO
    
from cda
where DT_COMPTC >= '{data_inicio}'
and DT_COMPTC <= '{data_fim}'
and FILE = 'PL'
""".format(data_inicio=data_inicio, data_fim=data_fim)
pl = pd.read_sql_query(sqlquery, engine, parse_dates=['DT_COMPTC'])

In [32]:
valor_tipo_ativos = carteira.groupby(['CNPJ_FUNDO','DENOM_SOCIAL','DT_COMPTC','TP_ATIVO'])['VL_MERC_POS_FINAL'].sum()

In [71]:
valor_pl = pl.groupby(['CNPJ_FUNDO','DENOM_SOCIAL','DT_COMPTC'])['VL_PATRIM_LIQ'].sum().replace(0, pd.np.nan)

In [73]:
pesos_aloc = peso_tipo_ativos.droplevel('DENOM_SOCIAL').unstack('DT_COMPTC')\
.div(valor_pl.droplevel('DENOM_SOCIAL').unstack('DT_COMPTC'))

In [100]:
x = peso_tipo_ativos.unstack('DT_COMPTC').loc[:,'2010':'2018'].mean(1).unstack('TP_ATIVO')

In [101]:
aloc_em_acoes = pesos_aloc.mean(1).unstack("TP_ATIVO")[['Ação ordinária','Ação preferencial']].sum(1)

In [102]:
cnpj_name_map = pl.set_index('CNPJ_FUNDO')['DENOM_SOCIAL'].to_dict()
cnpj_name_map = {k:(k,v) for k,v in cnpj_name_map.items()}

In [160]:
num_ativos = carteira.groupby(['CNPJ_FUNDO','DT_COMPTC','TP_ATIVO'])['TP_ATIVO'].count()

In [168]:
med_ativos = num_ativos.unstack('TP_ATIVO')[['Ação ordinária','Ação preferencial']].sum(1)

In [173]:
num_ativos = med_ativos.groupby(level='CNPJ_FUNDO').mean()

In [174]:
aloc_em_acoes[aloc_em_acoes.between(0.8,1)].sort_values().to_frame('aloc').join(num_ativos.to_frame('num'))\
.sort_values('num', ascending=False)

,aloc,num
CNPJ_FUNDO,,
11.184.136/0001-15,0.952933,197.433735
10.263.584/0001-41,0.956064,163.990654
07.586.804/0001-63,0.905861,163.910256
10.809.064/0001-91,0.976047,163.622222
11.455.378/0001-04,0.971331,161.203883
10.418.335/0001-88,0.958886,134.822430
13.966.586/0001-95,0.925367,128.964706
14.864.723/0001-43,0.951808,121.736111
10.406.511/0001-61,0.969429,120.943396


In [175]:
peso_tipo_ativos.xs('11.184.136/0001-15', level='CNPJ_FUNDO').unstack('DT_COMPTC')

DT_COMPTC                                                     2011-10-31  \
DENOM_SOCIAL                TP_ATIVO                                       
IT NOW IGCT FUNDO DE ÍNDICE Ação ordinária                    26399039.0   
                            Ação preferencial                 21338849.0   
                            Certificado de depósito de ações    886585.0   
                            Outros                               59468.0   
                            Recibo de subscrição                     NaN   
                            Contrato Futuro                          NaN   
                            Título público federal                   NaN   

DT_COMPTC                                                       2011-11-30  \
DENOM_SOCIAL                TP_ATIVO                                         
IT NOW IGCT FUNDO DE ÍNDICE Ação ordinária                    2.587393e+07   
                            Ação preferencial                 2.064825e+07   
                            Certificado de depósito de ações  8.097560e+05   
                            Outros                            6.562505e+04   
                            Recibo de subscrição                       NaN   
                            Contrato Futuro                            NaN   
                            Título público federal                     NaN   

DT_COMPTC                                                       2011-12-31  \
DENOM_SOCIAL                TP_ATIVO                                         
IT NOW IGCT FUNDO DE ÍNDICE Ação ordinária                    2.582250e+07   
                            Ação preferencial                 2.129006e+07   
                            Certificado de depósito de ações  8.697900e+05   
                            Outros                            3.412550e+05   
                            Recibo de subscrição              8.364900e+02   
                            Contrato Futuro                            NaN   
                            Título público federal                     NaN   

DT_COMPTC                                                       2012-01-31  \
DENOM_SOCIAL                TP_ATIVO                                         
IT NOW IGCT FUNDO DE ÍNDICE Ação ordinária                    2.544971e+07   
                            Ação preferencial                 2.097602e+07   
                            Certificado de depósito de ações  8.857620e+05   
                            Outros                            3.126659e+05   
                            Recibo de subscrição              3.184021e+05   
                            Contrato Futuro                   1.920000e+03   
                            Título público federal                     NaN   

DT_COMPTC                                                       2012-02-29  \
DENOM_SOCIAL                TP_ATIVO                                         
IT NOW IGCT FUNDO DE ÍNDICE Ação ordinária                    2.711370e+07   
                            Ação preferencial                 2.163559e+07   
                            Certificado de depósito de ações  1.006060e+06   
                            Outros                            3.380444e+05   
                            Recibo de subscrição              2.861686e+05   
                            Contrato Futuro                  -1.564000e+03   
                            Título público federal                     NaN   

DT_COMPTC                                                       2012-03-31  \
DENOM_SOCIAL                TP_ATIVO                                         
IT NOW IGCT FUNDO DE ÍNDICE Ação ordinária                    2.595265e+07   
                            Ação preferencial                 1.849861e+07   
                            Certificado de depósito de ações  1.854050e+05   
                            Outros                            7.891528e+05   
                            Recibo 

In [191]:
cnpj_fundo = '07.586.804/0001-63'
# carteira e pesos
sqlquery = """
select * from cda
where CNPJ_FUNDO = '{cnpj_fundo}'
and DT_COMPTC >= '{data_inicio}'
and DT_COMPTC <= '{data_fim}'
""".format(cnpj_fundo=cnpj_fundo, data_inicio=data_inicio, data_fim=data_fim)
carteira = pd.read_sql_query(sqlquery, engine, parse_dates=['DT_COMPTC'])
pl = carteira.sort_values(['DT_COMPTC', 'FILE'])[
    'VL_PATRIM_LIQ'].fillna(method='bfill')
carteira['VL_PATRIM_LIQ'] = pl
carteira['peso'] = carteira['VL_MERC_POS_FINAL'].div(carteira['VL_PATRIM_LIQ'])

carteira = carteira[carteira.FILE != 'PL']

# Valores a pagar são negativos
q = (carteira.TP_APLIC == 'Valores a pagar') | (
    carteira.TP_APLIC.str.lower().str.contains('obriga|lançad'))
carteira.loc[q, ['VL_MERC_POS_FINAL', 'peso']] *= -1

pivot_cols = carteira.CD_ATIVO\
    .fillna(carteira.CD_ISIN)\
    .fillna(carteira[carteira.FILE.str.contains('BLC_8') & carteira.TP_ATIVO.str.contains("Ação|Ações|Outr|Recibo|BDR", flags=re.IGNORECASE)].DS_ATIVO.str.extract(r'([A-Z]{4,5}\d+\d?)', expand=False))\
    .fillna(carteira[carteira.FILE.str.contains('BLC_8') & carteira.TP_ATIVO.str.contains("Opção|Opcões", flags=re.IGNORECASE)].DS_ATIVO.str.extract(r'([A-Z]{4,5}\d+\d?)', expand=False))\
    .fillna(carteira[carteira.TP_ATIVO.str.contains("Debênture", flags=re.IGNORECASE)].CD_INDEXADOR_POSFX)\
    .fillna(carteira[carteira.TP_APLIC.str.lower().isin(['disponibilidades', 'valores a pagar', 'valores a receber'])].TP_APLIC.str.upper())\
    .fillna(carteira[carteira.TP_ATIVO.str.contains('T.tulo p.blico federal', flags=re.IGNORECASE)].DS_ATIVO.str.extract("(BR..........)", expand=False))\
    .fillna(carteira[carteira.TP_ATIVO.str.contains('T.tulo p.blico federal', flags=re.IGNORECASE)].DS_ATIVO.str.extract(r'(\d{6})', expand=False))\
    .fillna(carteira.CNPJ_FUNDO_COTA)\
    .fillna(carteira.CD_ATIVO_BV_MERC)\
    .fillna(carteira[(carteira.TP_ATIVO.str.contains(r'Contrato Futuro.*', flags=re.IGNORECASE)) & (carteira.DS_ATIVO.str.contains('IND'))].DS_ATIVO.str.extract('(IND)', expand=False))\
    .fillna(carteira.DS_ATIVO)
print("Ativos sem identificação:", pivot_cols.isna().sum())

Ativos sem identificação: 0


In [192]:
x = carteira.pivot_table(index='DT_COMPTC', columns=pivot_cols, values='peso', aggfunc='sum')

In [ ]:
x[x>0].plot.area(legend=False)

In [198]:
cnpjs = """
11.184.136/0001-15
10.263.584/0001-41
07.586.804/0001-63
10.809.064/0001-91
11.455.378/0001-04
10.418.335/0001-88
13.966.586/0001-95
14.864.723/0001-43
10.406.511/0001-61
13.412.264/0001-02
02.547.088/0001-91
14.422.389/0001-78
11.905.308/0001-00
17.036.319/0001-70
10.406.600/0001-08
26.195.254/0001-04
21.407.758/0001-19
07.686.680/0001-98
18.820.186/0001-90
19.388.257/0001-90
03.370.003/0001-05
10.406.499/0001-95
10.355.506/0001-77
05.589.418/0001-08
13.902.338/0001-80
20.147.499/0001-71
14.816.867/0001-24
10.361.866/0001-81
40.428.039/0001-29
22.282.955/0001-11
18.206.957/0001-54
08.607.705/0001-83
07.685.000/0001-11
09.421.760/0001-47
01.989.897/0001-90
10.905.737/0001-07
18.098.502/0001-62
02.831.383/0001-75
19.465.180/0001-05
18.691.405/0001-89
15.586.774/0001-13
11.436.206/0001-85
09.531.769/0001-00
13.748.605/0001-07
08.606.341/0001-17
09.485.887/0001-20
07.665.283/0001-30
09.552.757/0001-62
06.245.549/0001-22
13.883.056/0001-83
11.119.089/0001-26
17.021.319/0001-04
11.702.061/0001-17
07.291.002/0001-26
15.614.285/0001-28
07.319.063/0001-54
07.325.714/0001-19
04.894.438/0001-11
07.194.559/0001-49
13.094.664/0001-09

"""
list(map(cnpj_name_map.get, cnpjs.split()))


[('11.184.136/0001-15', 'IT NOW IGCT FUNDO DE ÍNDICE'),
 ('10.263.584/0001-41', 'ITAÚ INDEX AÇÕES IBRX - FUNDO DE INVESTIMENTO'),
 ('07.586.804/0001-63', 'UNIBANCO T FUNDO DE INVESTIMENTO RENDA FIXA'),
 ('10.809.064/0001-91', 'BRAM H FUNDO DE INVESTIMENTO EM AÇÕES PASSIVO IBRX'),
 ('11.455.378/0001-04',
  'ISHARES IBRX - ÍNDICE BRASIL (IBRX-100) FUNDO DE ÍNDICE'),
 ('10.418.335/0001-88',
  'BB PREVIDENCIÁRIO AÇÕES GOVERNANÇA FUNDO DE INVESTIMENTO'),
 ('13.966.586/0001-95',
  'ÓRAMA BOLSA MID LARGE - SMALL CAP FUNDO DE INVESTIMENTO EM AÇÕES'),
 ('14.864.723/0001-43', 'TEXAS FUNDO DE INVESTIMENTO EM AÇÕES'),
 ('10.406.511/0001-61', 'ISHARES IBOVESPA FUNDO DE ÍNDICE'),
 ('13.412.264/0001-02',
  'ITAÚ INSTITUCIONAL BOLSA INDEX FUNDO DE INVESTIMENTO EM AÇÕES'),
 ('02.547.088/0001-91', 'FUNDO DE INVESTIMENTO DE AÇÕES BELLS'),
 ('14.422.389/0001-78', 'RT SOLSTICE AÇÕES - FUNDO DE INVESTIMENTO'),
 ('11.905.308/0001-00',
  'VEGAS RENDA FIXA CRÉDITO PRIVADO FUNDO DE INVESTIMENTO'),
 ('17.036.319

11.184.136/0001-15	0.952933	197.433735
10.263.584/0001-41	0.956064	163.990654
07.586.804/0001-63	0.905861	163.910256
10.809.064/0001-91	0.976047	163.622222
11.455.378/0001-04	0.971331	161.203883
10.418.335/0001-88	0.958886	134.822430
13.966.586/0001-95	0.925367	128.964706
14.864.723/0001-43	0.951808	121.736111
10.406.511/0001-61	0.969429	120.943396
13.412.264/0001-02	0.967112	120.292135
02.547.088/0001-91	0.963849	115.955224
14.422.389/0001-78	0.945188	115.888889
11.905.308/0001-00	0.945454	115.555556
17.036.319/0001-70	0.887821	114.560000
10.406.600/0001-08	0.927984	114.358491
26.195.254/0001-04	0.946761	111.545455
21.407.758/0001-19	0.965569	107.269231
07.686.680/0001-98	0.913402	105.588235
18.820.186/0001-90	0.962965	105.222222
19.388.257/0001-90	0.875288	102.559322
03.370.003/0001-05	0.956515	102.090323
10.406.499/0001-95	0.979432	101.647887
10.355.506/0001-77	0.859212	100.236842
05.589.418/0001-08	0.949131	98.548387
13.902.338/0001-80	0.962218	98.333333
20.147.499/0001-71	0.949130	97.962264
14.816.867/0001-24	0.892447	97.950000
10.361.866/0001-81	0.900547	97.685714
40.428.039/0001-29	0.934531	96.967742
22.282.955/0001-11	0.950429	96.292683
...	...	...
18.206.957/0001-54	0.982282	0.250000
08.607.705/0001-83	0.999464	0.250000
07.685.000/0001-11	0.999903	0.250000
09.421.760/0001-47	0.861090	0.241379
01.989.897/0001-90	0.967122	0.222222
10.905.737/0001-07	0.906725	0.210526
18.098.502/0001-62	0.850730	0.210526
02.831.383/0001-75	0.970519	0.200000
19.465.180/0001-05	0.967418	0.200000
18.691.405/0001-89	0.983674	0.191489
15.586.774/0001-13	0.977334	0.166667
11.436.206/0001-85	0.998920	0.166667
09.531.769/0001-00	0.854296	0.166667
13.748.605/0001-07	0.904605	0.166667
08.606.341/0001-17	0.996858	0.142857
09.485.887/0001-20	0.899154	0.101695
07.665.283/0001-30	0.999895	0.100000
09.552.757/0001-62	0.839939	0.096774
06.245.549/0001-22	0.996297	0.077419
13.883.056/0001-83	0.957501	0.076923
11.119.089/0001-26	0.995915	0.071429
17.021.319/0001-04	0.901721	0.056338
11.702.061/0001-17	0.999687	0.054545
07.291.002/0001-26	0.944995	0.043478
15.614.285/0001-28	0.981706	0.043478
07.319.063/0001-54	0.997888	0.043478
07.325.714/0001-19	0.998898	0.043478
04.894.438/0001-11	0.987301	0.029851
07.194.559/0001-49	0.997136	0.020408
13.094.664/0001-09	0.912871	0.018182